# Toy PGP
PGP (Pretty Good Privacy) is a software protocol that enables people to securely exchange information by encrypting, decrypting, and digitally signing data. PGP combines multiple cryptographic methods:

1.   Public-key cryptography: Each user has a public key (shared) and a private key (kept secret). Messages encrypted with the public key can only be decrypted with the private key, ensuring only the intended recipient can read the message
2.   Symmetric-key cryptography: Once a shared secret (session key) is established, symmetric encryption is used for fast data encryption and decryption.
3.   Hashing and digital signatures: PGP can create a hash of a message and sign it with the sender’s private key. This allows the recipient to verify the authenticity and integrity of the message using the public key.
    
PGP is widely used to secure email and file communications against unauthorized access and tampering. The protocol follows the OpenPGP standard (RFC 4880), and is interoperable with tools like GnuPG (Source: Fortinet)

---

For this project, we were asked to simulate a simplified PGP workflow using only basic algorithms:
1.   Send: encode (“hash”) + transpose + create a verification checksum
2.   Verify: recompute checksum and return one of three outcomes
3.   Decrypt: reverse transpose + reverse encoding to read plaintext
There are three outcomes that are shown in verification of a checksum:


*   Valid — checksum matches and key is trusted
*   Corrupted — checksum mismatch
*   Unknown sender — key not in trusted list


> CHECKSUM: A value calculated from a set of data used to verify its integrity. It plays a critical role in identifying errors during data transmission or storage, ensuring that the received data matches the original.




# SENDER A (SCRIPT 1)

In [5]:
private_key_A = 3
hash_transpose_step = 7
public_key_A = 34567
MOD = 100000

#This section allows Sender A to choose between two algorithmic flows.
print("Choose what to execute (Sending Message or Decrypt Message): ")
print("1. Send Message")
print("2. Decrypt Message")
choice = input("Enter your choice (1/2): ")

# loop logic for the choices
if choice == '1':
    message = input("Enter message to send: ")

#hash logic for option 1
    hashed_message = ""
    for char in message:
        hashed_code = (ord(char) + private_key_A) % 256   # ord(char) → converts each character into its ASCII number.
        hashed_message += chr(hashed_code)                # Add the private key (+3 in this case).
                                                          # % 256 ensures values stay between 0 and 255.
                                                          # chr() converts the shifted value back into a character.

    # Moves the first hash_transpose_step (7) characters to the end.
    transposed = hashed_message[hash_transpose_step:] + hashed_message[:hash_transpose_step] # [AI] AI use case to understand slicing

    checksum = 0        #initilaize the value in checksum variable to 0

    for char in transposed:                               # Loop through every character in the transposed message.
        checksum = (checksum + ord(char)) % MOD           # Add their ASCII values together, using % MOD to wrap the total.
    checksum = (checksum * public_key_A) % MOD            # Multiply by the sender’s public key to simulate a digital signature.

    print("\nCopy to Verifier or Sender (Verifier Script)")
    print("Hashed Message:", transposed)
    print("PGP Checksum:", checksum)
    print("Sender A Public Key: ", public_key_A)
    print("Sender A Private Key: ", private_key_A)
    print("Hash Transpose Step: ", hash_transpose_step)

  #Reply decrypting from other sender
elif choice == '2':

  #This branch reverses the encryption process
    transposed = input("Enter the transposed message: ")
    private_key = int(input("Enter the Private Key of other sender: "))
    transpose_step = int(input("Enter the Hash Transpose Step from other sender: "))

    if transpose_step > 0:
      decrypted_stage1 = transposed[-transpose_step:] + transposed[:-transpose_step]
      #Reverse logic of transposition
    else:
      decrypted_stage1 = transposed

    decoded_message = ''
    for char in decrypted_stage1:
        decoded_code = (ord(char) - private_key) % 256 # Subtract the private key from each ASCII value (the opposite of the encryption logic)
                                                       # % 256 keeps the result valid even if subtraction goes below 0
        decoded_message += chr(decoded_code)

    print("\nDecrypted Message:", decoded_message)

else:
    print("Invalid choice. Please enter 1 or 2.")

Choose what to execute (Sending Message or Decrypt Message): 
1. Send Message
2. Decrypt Message
Enter your choice (1/2): 1
Enter message to send: Hello There, General Kenobi

Copy to Verifier or Sender (Verifier Script)
Hashed Message: khuh/#Jhqhudo#NhqrelKhoor#W
PGP Checksum: 50809
Sender A Public Key:  34567
Sender A Private Key:  3
Hash Transpose Step:  7


# Verifier (SCRIPT 2)



In [6]:
known_keys = [34567,56789] #public keys of both senders
MOD = 100000 # MOD used for checksum calculations and protects program from overflows (checksum math within 0–99999)

transposed = input("Enter transposed message: ") # the hashed message string from either sender A or B
pgp_checksum = int(input("Enter PGP checksum: ")) # the numeric signature attached to the message
sender_public_key = int(input("Enter sender's public key: ")) # The sender's public key

computed = 0 # initialising the computed variable to start the decoding algorithm

for char in transposed: #This will keep looping as long as there are undecoded characters
  computed = (computed + ord(char)) % MOD # processes each character to calculate its ASCII value
computed = (computed * sender_public_key) % MOD # this produces the computed checksum, identical to what the sender generated if the message wasn’t altered

if sender_public_key not in known_keys:
  result = "Unknown Sender or Unverified Key"
elif computed == pgp_checksum:
  result = "Verified and Authentic PGP Checksum"
else:
  result = "Message Integrity compromised"

print("Verification Results: ", result)

Enter transposed message: khuh/#Jhqhudo#NhqrelKhoor#W
Enter PGP checksum: 50809
Enter sender's public key: 34567
Verification Results:  Verified and Authentic PGP Checksum


# Sender B (Script 3)



In [7]:
# Everything about sender B is in reverse hence we have structures same in the script

private_key_B = 5
hash_transpose_step_B = 4
public_key_B = 56789 #new fixed key
MOD = 100000

# Menu to decide on choice
print("Choose what to execute (Sending Message or Decrypt Message): ")
print("1. Decrypt Message")
print("2. Send Message")
choice = input("Enter your choice (1/2): ")

if choice == '1':
    transposed = input("Enter the hashed message: ")
    private_key = int(input("Enter the Private Key of other sender: "))
    transpose_step = int(input("Enter the Hash Transpose Step from other sender: "))

    if transpose_step > 0:
      decrypted_stage1 = transposed[-transpose_step:] + transposed[:-transpose_step]
    else:
      decrypted_stage1 = transposed

    decoded_message = ''
    for char in decrypted_stage1:
        decoded_code = (ord(char) - private_key) % 256
        decoded_message += chr(decoded_code)

    print("\nDecrypted Message:", decoded_message)

# cipher logic for option 2
elif choice == '2':
    message = input("Enter reply message to send: ")

    hashed_message = ""
    for char in message:
        hashed_code = (ord(char) + private_key_B) % 256
        hashed_message += chr(hashed_code)

    transposed = hashed_message[hash_transpose_step_B:] + hashed_message[:hash_transpose_step_B]

    checksum = 0
    for char in transposed:
        checksum = (checksum + ord(char)) % MOD
    checksum = (checksum * public_key_B) % MOD

    print("\nCopy to Verifier or sender")
    print("Transposed Message:", transposed)
    print("PGP Checksum:", checksum)
    print("Sender A Public Key: ", public_key_B)
    print("Sender A Private Key: ", private_key_B)
    print("Hash Transpose Step: ", hash_transpose_step_B)

else:
    print("Invalid choice. Please enter 1 or 2.")

Choose what to execute (Sending Message or Decrypt Message): 
1. Decrypt Message
2. Send Message
Enter your choice (1/2): 1
Enter the hashed message: khuh/#Jhqhudo#NhqrelKhoor#W
Enter the Private Key of other sender: 3
Enter the Hash Transpose Step from other sender: 7

Decrypted Message: Hello There, General Kenobi
